In [28]:
# Criar uma lista lambda com todas as combinações possíveis de a, b, x e y
lambda_list = [(lambda a, b, x, y: (a, b, x, y))(a, b, x, y) for a in range(2) for b in range(2) for x in range(2) for y in range(2)]

print("lambda_list:", lambda_list)
# Inicializar lista para armazenar os índices onde a = b = 0
indices_a_b_0 = []

# Percorrer a lista lambda e verificar as condições
for i, tup in enumerate(lambda_list):
    a, b, _, _ = tup
    if a == b == 0:
        indices_a_b_0.append(i)

print("Índices onde a = b = 0:", indices_a_b_0)
# Inicializar lista para armazenar os índices onde a ⊕ b = x * y
indices_xor_condition = []

# Percorrer a lista lambda e verificar as condições
for i, tup in enumerate(lambda_list):
    a, b, x, y = tup
    if a ^ b == x * y:
        indices_xor_condition.append(i)

print("Índices onde a ⊕ b = x * y:", indices_xor_condition)
# Inicializar vetor p_L com 16 componentes
p_L = [0] * 16

# Definir p_L[i] = 1 para os índices contidos em indices_a_b_0
for i in indices_a_b_0:
    p_L[i] = 1

print("p_L:", p_L)
# Inicializar vetor p_PR com 16 componentes
p_PR = [0] * 16

# Definir p_L[i] = 1 para os índices contidos em indices_a_b_0
for i in indices_xor_condition:
    p_PR[i] = 1

print("p_PR:", p_PR)
# Criar vetor com 16 componentes iguais a 1/4
p_I = [1/4] * 16

print("p_I:", p_I)



lambda_list: [(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 1, 0), (0, 1, 1, 1), (1, 0, 0, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 0, 1, 1), (1, 1, 0, 0), (1, 1, 0, 1), (1, 1, 1, 0), (1, 1, 1, 1)]
Índices onde a = b = 0: [0, 1, 2, 3]
Índices onde a ⊕ b = x * y: [0, 1, 2, 7, 11, 12, 13, 14]
p_L: [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
p_PR: [1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0]
p_I: [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]


In [337]:
# Função p(alpha,beta)
import numpy as np
import cvxpy as cp

def func_p(alpha,beta):
    p_PR = np.array([1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0])
    p_PL = np.array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    p_I = np.array([0.25] * 16)
    p = alpha * p_PR + (1 - alpha) * (beta * p_L + (1 - beta) * p_I)
    # Convertendo para um array numpy
    p = np.array(p)
    
    
    return p


#p_alpha_beta = func_(alpha, beta)

In [421]:
import numpy as np
import cvxpy as cp

def func_marg(p_alpha_beta):
    # Criar uma lista lambda com todas as combinações possíveis de a, b, x e y
    lambda_list = [(lambda a, b, x, y: (a, b, x, y))(a, b, x, y) for a in range(2) for b in range(2) for x in range(2) for y in range(2)]
    # Função para encontrar o índice correspondente na lista lambda_list
    def f(a, b, x, y):
        for i, tupla in enumerate(lambda_list):
            if tupla == (a, b, x, y):
                return i
        # Se não encontrar, retornar -1 ou levantar uma exceção, dependendo do caso de uso
        return -1

    # Lista para armazenar os resultados
    p_marg = []

    # Percorrendo a lista e calculando os valores para p_marg
    for i, (a, b, x, y) in enumerate(lambda_list):
        # Trocando os valores de a e b simultaneamente
        a, b = 1 - a, 1 - b
        # Calculando o vetor p de marginalização p_marg
        somaby = 0
        somaax = 0
        for b_iter in range(2):  # b_iter = 0 até 1
            for y_iter in range(2):  # y_iter = 0 até 1
                somaby += p_alpha_beta[f(a, b_iter, x, y_iter)]
        for a_iter in range(2):  # a_iter = 0 até 1
            for x_iter in range(2):  # x_iter = 0 até 1
                somaax += p_alpha_beta[f(a_iter, b, x_iter, y)]
        prod = somaby * somaax
        # Adicionando o resultado à lista p_marg
        p_marg.append(prod)

    
    return p_marg

# Exemplo de uso da função
# p_marg = func_marg(lambda_list, p_alpha_beta)



In [339]:
# Import packages.
import cvxpy as cp
import numpy as np

beta = 0.3
alpha = cp.Variable()
p = func_p(1,beta)
p2 = func_marg(p)

print(p,p2)

    




[1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0.] [4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]


In [460]:
import cvxpy as cp
import numpy as np

# Defina os tamanhos dos vetores p e p1
n = 16

# Defina o número de passos para variar beta
num_steps = 20

# Defina o intervalo para beta
beta_values = np.linspace(0, 1, num_steps)

# Faça um loop para variar beta
for beta_value in beta_values:
    # Defina as variáveis do problema
    alpha = cp.Variable()

    # Defina p e p1 em termos de alpha e beta
    p = alpha * p_PR + (1 - alpha) * (beta_value * p_L + (1 - beta_value) * p_I)
    p_marg = func_marg(p)

    # Defina a função objetivo
    objective = cp.Minimize(cp.norm(p - p_marg, 'fro'))
    # Defina as restrições
    constraints = [alpha >= 0, alpha <= 1]

    # Crie o problema de otimização
    problem = cp.Problem(objective, constraints)

    # Resolva o problema
    problem.solve()

    # Verifique o status da solução
    if problem.status == cp.OPTIMAL:
        # Obtenha o valor ótimo de alpha
        optimal_alpha = alpha.value
        print(f"Para beta = {beta_value}, valor ótimo de alpha:", optimal_alpha)
        p_value = optimal_alpha * p_PR + (1 - optimal_alpha) * (beta_value * p_L + (1 - beta_value) * p_I)
        p_marg = func_marg(p_value)
        if np.allclose(p_value, p_marg):
            print("CAVALO")
    else:
        print("O problema não foi resolvido de forma ótima.")


ValueError: The input must be a single CVXPY Expression, not a list. Combine Expressions using atoms such as bmat, hstack, and vstack.